In [2]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
# prepare data
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [5]:
# scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [6]:
# model
class Model(nn.Module):
    def __init__(self, n_input_features):
        # super(Model, self).__init__() #python2
        super().__init__() #python3
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

model = Model(n_features)

In [7]:
# loss and optimizer
num_epochs = 100
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [11]:
X_train.shape

torch.Size([455, 30])

In [12]:
for epoch in range(num_epochs):
    # Forward pass and loss
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    # loss = criterion(y_train, y_pred) #this will cause error

    # Backward pass and update
    loss.backward()
    optimizer.step()

    # zero grad before new step
    optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

epoch: 10, loss = 0.2325
epoch: 20, loss = 0.2239
epoch: 30, loss = 0.2162
epoch: 40, loss = 0.2092
epoch: 50, loss = 0.2029
epoch: 60, loss = 0.1971
epoch: 70, loss = 0.1918
epoch: 80, loss = 0.1869
epoch: 90, loss = 0.1823
epoch: 100, loss = 0.1781


In [13]:
with torch.no_grad():
    y_pred = model(X_test)
    y_pred_cls = y_pred.round()
    acc = y_pred_cls.eq(y_test).sum()/float(y_test.shape[0])
print(f'accuracy = {acc:.4f}')

accuracy = 0.9035


In [16]:
with torch.no_grad():
    y_pred = model(X_test) #0~1
    # print(y_pred)
    y_pred_cls = y_pred.round()#0 or 1
    # print(y_pred_cls)
    acc = y_pred_cls.eq(y_test).sum()/float(y_test.shape[0]) #eq : equal ==

tensor([[8.3717e-01],
        [9.8473e-01],
        [4.5856e-01],
        [9.3550e-01],
        [9.0452e-01],
        [5.6762e-01],
        [5.4207e-01],
        [9.7574e-01],
        [1.4888e-02],
        [1.7574e-02],
        [3.1218e-02],
        [7.9786e-01],
        [8.4007e-01],
        [6.2224e-01],
        [8.9764e-01],
        [3.2294e-04],
        [9.6589e-01],
        [9.7745e-01],
        [8.7948e-01],
        [1.3367e-01],
        [8.5661e-01],
        [2.0684e-01],
        [3.9043e-03],
        [6.6241e-04],
        [3.6564e-02],
        [8.3831e-01],
        [4.2072e-01],
        [9.3673e-01],
        [9.8425e-01],
        [9.6878e-01],
        [8.8298e-01],
        [9.4770e-01],
        [2.8540e-02],
        [7.8010e-01],
        [8.8237e-01],
        [5.9443e-01],
        [9.7989e-01],
        [2.2628e-01],
        [7.2908e-01],
        [9.4922e-01],
        [9.9473e-02],
        [9.0804e-01],
        [3.7919e-02],
        [7.6127e-01],
        [7.7916e-01],
        [9

In [22]:
p = iter(model.parameters())
print(next(p))

Parameter containing:
tensor([[-0.1212, -0.2101, -0.0530, -0.1893, -0.2601, -0.2801,  0.0329, -0.3523,
         -0.0528,  0.0577, -0.2340, -0.0252, -0.2477, -0.0945,  0.0734, -0.0256,
         -0.0999, -0.1536,  0.0491,  0.1071, -0.2880, -0.2101, -0.2960, -0.1971,
          0.0147, -0.1101, -0.1868, -0.3494, -0.0549,  0.0188]],
       requires_grad=True)
